# Investment Assignment

Data Analysis to find the best suitable Investment Horizon for **Spark Funds**

## 1. Investment Type Analysis

Finding the Best Investment Type for Spark Funds

In [ ]:
import pandas

Loading the **rounds2.csv** file using pandas. 
The length of the column **company_permalink** must be the number of unique companies in it.

In [ ]:
rounds2 = pandas.read_csv('rounds2.csv', sep=',', encoding='unicode_escape')
rounds2_permalinks = rounds2['company_permalink'].str.lower()
rounds2_unique_permalinks = set(rounds2_permalinks)
print(f'Length of rounds2 Unique permalink : {len(rounds2_unique_permalinks)}')

Repeating the same steps above for **companies.csv**

In [ ]:
companies = pandas.read_csv('companies.csv', sep='\t', encoding='unicode_escape')
companies_permalinks = companies['permalink'].str.lower()
companies_unique_permalinks = set(companies_permalinks)
print(f'Length of comapnies Unique permalinks : {len(companies_unique_permalinks)}')

In [ ]:
difference = rounds2_unique_permalinks.difference(companies_unique_permalinks)   # set difference
print(f'Number of companies that are NOT in companies.csv but in rounds2.csv = {len(difference)}')

Converting every value to lower case of column **company_permalink** of rounds2 and renaming as **permalink**

In [ ]:
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
rounds2 = rounds2.rename({'company_permalink': 'permalink'}, axis=1)
rounds2.head()

Converting every value to lower case of column **permalink** of companies.

In [ ]:
companies['permalink'] = companies['permalink'].str.lower()
companies.head()

Merging the two dataframes together as **master**

In [ ]:
companies.set_index(['permalink'])
rounds2.set_index(['permalink'])
master = pandas.merge(companies, rounds2,how='inner', on='permalink')
master.head()

In [ ]:
print(f'Number of records in master dataframe = {len(master)}')

Finding **The most representing value** for the following fund types.
1. Venture fund
2. Angel fund
3. Seed fund
4. Private Equity fund.

Note : All values are rounded off to two decimal places.

In [ ]:
venture_fund_mean = master.loc[master.funding_round_type=='venture'].raised_amount_usd.mean()
angel_fund_mean = master.loc[master.funding_round_type=='angel'].raised_amount_usd.mean()
seed_fund_mean = master.loc[master.funding_round_type=='seed'].raised_amount_usd.mean()
private_equity_fund_mean = master.loc[master.funding_round_type=='private_equity'].raised_amount_usd.mean()

print(f'Venture fund average : {round(venture_fund_mean, 2)}')
print(f'Angel fund average : {round(angel_fund_mean, 2)}')
print(f'Seed fund average : {round(seed_fund_mean, 2)}')
print(f'Private Equity fund average : {round(private_equity_fund_mean, 2)}')

**Since Spark Funds is ready to Invest between 5 to 10 million USD, _Venture Funding Type_ looks most suitable for them**

## 2. Country Analysis

From now, We'll use only venture type investment data for further analysis.

In [ ]:
master = master.loc[master.funding_round_type=='venture']
master.head()

The Top 9 Countries with **Highest Total Venture Funding** is listed below.

In [ ]:
highest_funded_countries = master.groupby(['country_code'])['raised_amount_usd'].sum()
highest_funded_countries = highest_funded_countries.sort_values(ascending=False).iloc[:9]
highest_funded_countries

Creating a new dataframe **top9** with details of Venture funding data in the top 9 countries.

In [ ]:
top9 = master.loc[master['country_code'].apply(lambda x: x in highest_funded_countries)]
top9.head(10)

## 3. Sector Analysis

Let's read the **mapping.csv** file into a dataframe and see how it looks.

In [ ]:
mapping_raw = pandas.read_csv('mapping.csv', index_col='category_list')
mapping_raw.head()

Let's reverse the one-hot encoding for an easier analysis

In [ ]:
mapping = pandas.DataFrame(pandas.get_dummies(mapping_raw).idxmax(1))
mapping.columns = ['main_sector']
mapping.index.names = ['primary_sector']
mapping.head()

Since we've discussed with the CEO and formed a business rule that **the first string before the vertical bar will be considered the primary sector**, we'll ignore the rest of the values in the **category_list** column in the master dataframe. We'll edit the master dataframe such that the primary sector according to the business rule will be there in the new column **primary_sector**.

In [ ]:
master.category_list = master.category_list.astype(str)
master['primary_sector'] = master.category_list.apply(lambda x: x.split('|')[0])
master.head()

Now let's merge the master data with the mapping data

In [ ]:
master = pandas.merge(master, mapping, how='inner', on='primary_sector')
master.head(10)

Now we have the master dataframe with two additional columns **primary_sector** and **main_sector**. We can do detailed analysis with respect to sectors which the startups work in.